In [23]:
import json
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import torch
from transformers import RobertaTokenizer, RobertaModel

### For Macs

In [24]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


### Defining Functions

In [33]:
def read_json_array(path):
  data = []
  with open(path, 'r') as file:
    for line in file:
      json_object = json.loads(line.strip())
      data.append(json_object)
  return data

# Function to generate embeddings
def generate_embeddings(texts, tokenizer, model, batch_size=8):
    """
    Generate embeddings for a list of texts using DistilRoBERTa with mean pooling.
    """
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model = model.to(device)

    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors='pt', max_length=512)
        inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to device

        with torch.no_grad():
            outputs = model(**inputs)
            # Use mean pooling for sentence representation
            # Exclude padding tokens by creating a mask
            attention_mask = inputs['attention_mask']
            token_embeddings = outputs.last_hidden_state  # Shape: (batch_size, seq_length, hidden_size)

            # Compute mean of embeddings, taking into account the attention mask
            masked_embeddings = token_embeddings * attention_mask.unsqueeze(-1)  # Shape: (batch_size, seq_length, hidden_size)
            # Sum embeddings and count non-padding tokens
            sum_embeddings = masked_embeddings.sum(dim=1)  # Shape: (batch_size, hidden_size)
            count_embeddings = attention_mask.sum(dim=1)  # Shape: (batch_size)

            # Avoid division by zero and compute mean
            mean_embeddings = sum_embeddings / count_embeddings.unsqueeze(-1).clamp(min=1e-9)  # Shape: (batch_size, hidden_size)
            embeddings.append(mean_embeddings)

    return torch.cat(embeddings, dim=0)

def reduce_dimensionality(embeddings, n_components=50):
    """
    Reduce the dimensionality of embeddings using PCA.
    """
    embeddings_np = embeddings.cpu().numpy()  # Move tensor to CPU and convert to NumPy
    pca = PCA(n_components=n_components)
    reduced_embeddings = pca.fit_transform(embeddings_np)
    return reduced_embeddings

def perform_clustering(embeddings, n_clusters=5):
    """
    Cluster embeddings using KMeans.
    """
    reduced_embeddings = reduce_dimensionality(embeddings)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(reduced_embeddings)
    return clusters


### Loading dataset

In [34]:
dataset_path = '../datasets/amazon_reviews/'
data_path = dataset_path + "Musical_Instruments.json"
meta_path = dataset_path + "meta_Musical_Instruments.json"

data = read_json_array(data_path)
# Extract relevant fields for processing
df = pd.DataFrame(data)
df['text'] = df['reviewText']

In [35]:
# Taking only first 1000 for demo 
df = df.head(1000)

In [36]:
len(df)

1000

### Creating Embeddings

In [37]:
# Initialize distilroberta tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')
model = RobertaModel.from_pretrained('distilroberta-base')
# Generate embeddings
print("Generating embeddings...")
embeddings = generate_embeddings(df['text'].tolist(), tokenizer, model)
print("Embeddings generated!")


Generating embeddings...
Embeddings generated!


### Clustering 

In [38]:
print("Reducing dimensionality and clustering...")
clusters = perform_clustering(embeddings)
df['cluster'] = clusters
print("Clustering complete!")

Reducing dimensionality and clustering...
Clustering complete!


In [39]:

# Visualizing or interpreting the clusters
print("Cluster distribution:")
print(df['cluster'].value_counts())

# Save results to a CSV for further analysis
df[['text', 'cluster']].to_csv('../outputs/amazon_reviews/clustered_reviews.csv', index=False)
print("Results saved to clustered_reviews.csv")

Cluster distribution:
cluster
2    448
0    196
3    196
1     92
4     68
Name: count, dtype: int64
Results saved to clustered_reviews.csv
